In [1]:
from pcomp_utils.kafka_handlers import KafkaProducerHandler, KafkaConsumerHandler

BROKERS = ["kafka:29092"]
KAFKA_BROKER = 'kafka:29092'

In [9]:
#from kafka import KafkaProducer, KafkaConsumer
import json
import threading
import time
import uuid


KAFKA_BROKER = 'kafka:9092'

def produce_messages():
    # producer = KafkaProducer(
    #     bootstrap_servers=BROKERS,
    #     value_serializer=lambda v: json.dumps(v).encode('utf-8')
    # )
    producer = KafkaProducerHandler(KAFKA_BROKER)

    for i in range(5):
        message = {
            "user_id": i,
            "action": "activate",
            "timestamp": "2025-03-12T12:00:00"
        }
        producer.send('activate-layer', message, 0)
        print("Message sent: ", message)
        time.sleep(1)

    producer.close()

def consume_messages():
    consumer = KafkaConsumerHandler('activate-layer', KAFKA_BROKER, partition=0)

    print("Starting consumer...")
    count = 0
    for message in consumer.consume():
        time.sleep(2)
        layer= message.get("layer")
        print(f"Received message: {layer}")
        count += 1
        if count >= 5:
            break

if __name__ == "__main__":
    # Start consumer in a separate thread
    consumer_thread = threading.Thread(target=consume_messages)
    consumer_thread.start()

    # Ensure the consumer is fully ready
    time.sleep(2)

    produce_messages()

    consumer_thread.join()


Starting consumer...
Message sent:  {'user_id': 0, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message: layer_0
Message sent:  {'user_id': 1, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 2, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message: layer_0
Message sent:  {'user_id': 3, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 4, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message: layer_0
Received message: layer_0
Received message: layer_0


In [3]:
import time
from pcomp_utils.kafka_confluent_utils import KafkaProducerHandler, KafkaConsumerHandler

KAFKA_BROKER = 'kafka:9092'
TOPIC_NAME = 'activate-layer'
GROUP_ID = 'default-group'

# Simple Kafka Producer
def simple_producer():
    producer = KafkaProducerHandler(KAFKA_BROKER)
    for i in range(10):
        message = f'This is message number {i}'
        producer.send(TOPIC_NAME, message)
        print(f'Produced: {message}')
        time.sleep(1)
    producer.close()

# Simple Kafka Consumer
def simple_consumer():
    consumer = KafkaConsumerHandler(TOPIC_NAME, KAFKA_BROKER, group_id=GROUP_ID)
    print('Starting consumer...')
    try:
        for message in consumer.consume():
            print(f'Consumed: {message}')
    except KeyboardInterrupt:
        print("Consumer interrupted")
    finally:
        consumer.commit()
        consumer.close()

if __name__ == '__main__':
    from threading import Thread

    # Start the consumer in a separate thread
    consumer_thread = Thread(target=simple_consumer)
    consumer_thread.start()

    # Give the consumer a moment to start
    time.sleep(2)

    # Run the producer
    simple_producer()

    # Wait for the consumer to finish
    consumer_thread.join()


Starting consumer...
Produced: This is message number 0
Produced: This is message number 1
Produced: This is message number 2
Consumed: This is message number 2
Produced: This is message number 3
Consumed: This is message number 3
Produced: This is message number 4
Consumed: This is message number 4
Produced: This is message number 5
Consumed: This is message number 5
Produced: This is message number 6
Consumed: This is message number 6
Produced: This is message number 7
Consumed: This is message number 7
Produced: This is message number 8
Consumed: This is message number 8
Produced: This is message number 9
Consumed: This is message number 9


In [1]:
consumer = KafkaConsumerHandler(TOPIC_NAME, KAFKA_BROKER, group_id=GROUP_ID)

topic = "activate-layer"

metadata = consumer.list_topics(topic)
partitions = [p.id for p in metadata.topics[topic].partitions.values()]

# Assign partitions manually
topic_partitions = [TopicPartition(topic, partition) for partition in partitions]
consumer.assign(topic_partitions)

# Seek to end (latest offset)
consumer.seek_to_end()

# Get the end offsets
end_offsets = consumer.position(topic_partitions)

# Commit those offsets
consumer.commit(offsets=end_offsets, asynchronous=False)

consumer.close()

print(f"Offsets reset to latest: {end_offsets}")

NameError: name 'KafkaConsumerHandler' is not defined

In [11]:
import threading
import time
from confluent_kafka import Producer, Consumer

TOPIC = "activate-layer"
BOOTSTRAP_SERVERS = "kafka:9092"

def produce_messages():
    producer_conf = {'bootstrap.servers': BOOTSTRAP_SERVERS}
    producer = Producer(producer_conf)

    def delivery_report(err, msg):
        if err:
            print(f"Delivery failed for message {msg.value()}: {err}")
        else:
            key = msg.key().decode('utf-8') if msg.key() else None
            print(f"Produced message with key={key} to partition {msg.partition()}")

    # A batch of messages using two keys.
    messages = [
        ("key1", "Message 1 for key1"),
        ("key2", "Message 1 for key2"),
        ("key1", "Message 2 for key1"),
        ("key2", "Message 2 for key2"),
        ("key1", "Message 3 for key1")
    ]

    for key, value in messages:
        producer.produce(TOPIC, key=key.encode('utf-8'), value=value.encode('utf-8'),
                         callback=delivery_report)
        producer.poll(0)  # Trigger delivery callback.
        time.sleep(0.2)   # Short pause between messages.

    producer.flush()
    print("Finished producing messages.\n")

def consume_messages(consumer_id, expected_key, group_id, run_time=10):
    # Using a unique group id for each consumer ensures they receive all messages.
    consumer_conf = {
        'bootstrap.servers': BOOTSTRAP_SERVERS,
        'group.id': group_id,
        'auto.offset.reset': 'earliest'
    }
    consumer = Consumer(consumer_conf)
    consumer.subscribe([TOPIC])
    print(f"Consumer {consumer_id} started in group '{group_id}' (expecting key '{expected_key}')")

    start_time = time.time()
    try:
        while time.time() - start_time < run_time:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer {consumer_id} error: {msg.error()}")
                continue

            actual_key = msg.key().decode('utf-8') if msg.key() else None
            value = msg.value().decode('utf-8')

            if actual_key == expected_key:
                print(f"Consumer {consumer_id} processed: {value} | key={actual_key} | partition={msg.partition()} | offset={msg.offset()}")
            else:
                # You can choose to not print anything if key doesn't match,
                # or log that it's ignoring this message.
                print(f"Consumer {consumer_id} ignoring message with key '{actual_key}'.")
            consumer.commit(msg)
    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()
        print(f"Consumer {consumer_id} closed.")

if __name__ == "__main__":
    # Each consumer gets its own consumer group so they each receive all messages.
    consumer_thread1 = threading.Thread(target=consume_messages, args=(1, "key1", "consumer_group_key1"))
    consumer_thread2 = threading.Thread(target=consume_messages, args=(2, "key2", "consumer_group_key2"))

    consumer_thread1.start()
    consumer_thread2.start()

    # Give consumers time to start and subscribe.
    time.sleep(1)

    produce_messages()

    # Allow some time for consumers to process the messages.
    time.sleep(5)

    consumer_thread1.join()
    consumer_thread2.join()

    print("All consumers have finished processing.")


Consumer 1 started in group 'consumer_group_key1' (expecting key 'key1')
Consumer 2 started in group 'consumer_group_key2' (expecting key 'key2')
Produced message with key=key1 to partition 1
Produced message with key=key2 to partition 3
Produced message with key=key1 to partition 1
Produced message with key=key2 to partition 3
Produced message with key=key1 to partition 1
Finished producing messages.

Consumer 2 ignoring message with key 'key1'.
Consumer 2 ignoring message with key 'key1'.
Consumer 2 ignoring message with key 'key1'.
Consumer 1 processed: Message 1 for key1 | key=key1 | partition=1 | offset=29753
Consumer 1 processed: Message 2 for key1 | key=key1 | partition=1 | offset=29754
Consumer 1 processed: Message 3 for key1 | key=key1 | partition=1 | offset=29755
Consumer 1 ignoring message with key 'key2'.
Consumer 1 ignoring message with key 'key2'.
Consumer 2 processed: Message 1 for key2 | key=key2 | partition=3 | offset=27
Consumer 2 processed: Message 2 for key2 | key=k

In [38]:
producer = KafkaProducerHandler(KAFKA_BROKER, "layer-0-streams")

In [39]:
neuron = 0
cnt = 0.0
num = 2.423847923324892
for i in range(128):
    producer.send_with_key("-1", f"{neuron}|{num + cnt}")
    neuron+=1
    cnt+=1.0

In [40]:
producer.close()